# Dataset split

This file divides our dataset into train, validation and test set. We can't use preprogrammed functions to do that, because we divided each of our files into other smallers (i.e. one sound to six images). Putting images made out of same mp3 file might lead to the *data leakage* and make our results *not trustworthy* and *biased*.

In [71]:
import os

# define number of files for all sets
train = 0.8 # 80% of all sound should be in the train set
val = 0.1 
test = 0.1
kfolds = 1

# first find all of the mp3 files in the directory
birds=[] # list of all birds (Ember, Phyll...)
blist=[] # list of files for one bird
flist=[] # list of all files for all birds. A list of blists. 
for root, dirs, files in os.walk("../data/xeno-canto-dataset-full/"):
    if root == "../data/xeno-canto-dataset-full/":
        birds=dirs
print(birds)

trainSet=[]
testSet=[]
valSet=[]

birdsShort=[]
i50=0
for nr,bird in enumerate(birds):
    for root, dirs, files in os.walk("../data/xeno-canto-dataset-full/"+bird):
        for file in files:
            if file.endswith(".mp3"):
                blist.append(os.path.join(root, file))
    if len(blist) > 50:
        birdsShort.append(str(i50)+bird[:5])
        i50 = i50+1;
        print("Found ", len(blist), ' mp3 files for ', bird)
        #i50 = i50+1;
        #print(i50, ". Found ", len(blist), ' files for ', bird ,'(',i+1,')')
        #birds50.append(bird)
        #flist.append(blist)
        trainSet.append(int(train*len(blist)))
        valSet.append(int(val*len(blist)))
        roundDiff = len(blist)-(int(train*len(blist))+int(test*len(blist))+int(val*len(blist))) 
        testSet.append(int(test*len(blist)) + roundDiff)
        print("Size of train: ", int(train*len(blist)),", val: ",int(val*len(blist)) ,", test: ",int(test*len(blist)))
        flist.append(blist)
    blist=[]


print(trainSet)
print(valSet)
print(testSet)


['Parusmajor', 'Turduspilaris', 'Passerdomesticus', 'Luscinialuscinia', 'Dendrocoposmajor', 'Corvusmonedula', 'Phoenicurusphoenicurus', 'Erithacusrubecula', 'Picapica', 'Phoenicurusochruros', 'Garrulusglandarius', 'Passermontanus', 'Corvuscorone', 'Corvusfrugilegus', 'Coccothraustescoccothraustes', 'Sittaeuropaea', 'Alaudaarvensis', 'Streptopeliadecaocto', 'Phylloscopustrochilus', 'Delichonurbicum', 'Turdusphilomelos', 'Phylloscopuscollybita', 'Fringillacoelebs', 'Sturnusvulgaris', 'Emberizacitrinella', 'Columbapalumbus', 'Troglodytestroglodytes', 'Paruscaeruleus', 'Apusapus', 'Cardueliscarduelis', 'Chlorischloris', 'Motacillaalba', 'Turdusmerula']
Found  418  mp3 files for  Parusmajor
Size of train:  334 , val:  41 , test:  41
Found  59  mp3 files for  Passerdomesticus
Size of train:  47 , val:  5 , test:  5
Found  107  mp3 files for  Luscinialuscinia
Size of train:  85 , val:  10 , test:  10
Found  111  mp3 files for  Phoenicurusphoenicurus
Size of train:  88 , val:  11 , test:  11
F

In [58]:
# randomly choose mp3 files for each set  

from random import sample

trainFiles=[]
valFiles=[]
testFiles=[]

for index, blist in enumerate(flist):
    randFiles = sample(range(len(blist)), len(blist))
    start = 0
    end = trainSet[index]
    trainFiles.append(randFiles[start:end])
    start = end 
    end = start + valSet[index]
    valFiles.append(randFiles[start:end])
    start = end 
    end = start + testSet[index] 
    testFiles.append(randFiles[start:end])
    print("Selected random files number:\n train: ", len(trainFiles[index]),"/",trainSet[index],
          ", val: ",len(valFiles[index]),"/",valSet[index],
          ", test: ",len(testFiles[index]),"/",testSet[index])



Selected random files number:
 train:  334 / 334 , val:  41 / 41 , test:  43 / 43
Selected random files number:
 train:  47 / 47 , val:  5 / 5 , test:  7 / 7
Selected random files number:
 train:  85 / 85 , val:  10 / 10 , test:  12 / 12
Selected random files number:
 train:  88 / 88 , val:  11 / 11 , test:  12 / 12
Selected random files number:
 train:  356 / 356 , val:  44 / 44 , test:  46 / 46
Selected random files number:
 train:  64 / 64 , val:  8 / 8 , test:  8 / 8
Selected random files number:
 train:  107 / 107 , val:  13 / 13 , test:  14 / 14
Selected random files number:
 train:  84 / 84 , val:  10 / 10 , test:  11 / 11
Selected random files number:
 train:  172 / 172 , val:  21 / 21 , test:  23 / 23
Selected random files number:
 train:  451 / 451 , val:  56 / 56 , test:  57 / 57
Selected random files number:
 train:  251 / 251 , val:  31 / 31 , test:  32 / 32
Selected random files number:
 train:  292 / 292 , val:  36 / 36 , test:  37 / 37
Selected random files number:
 tra

# Copying files to a new directory
In previous blocks we've randomly selected which files will belong to which sets. Now we have to find corresponding images to those files and copy them to new directory.

* new directory - i.e. ..data/mels/final
    - train 
        - one folder for each class
        - Ember
        - Phyll
        - ...
    - val
        - folders for each class
    - test
        - folders for each class

In [59]:
def extractName(string):
    return string.rsplit('/',1)[1].replace(' ', '')[:-4]

In [60]:
# sort all the lists to make copying files easier
sets=[trainFiles,valFiles,testFiles]
for fileSet in sets:
    for index,files in enumerate(fileSet):
        fileSet[index].sort()

In [72]:
# change full names to short

for root, dirs, files in os.walk("../data/xeno-canto-dataset-full/"):
    if root == "../data/xeno-canto-dataset-full/":
        birds=dirs
#birdsShort=[]
#for bird in birds:
#    birdsShort.append(bird[:5])

setNames = ["train/","val/","test/"]

print("Long: ", birds,"\nShort: ",birdsShort)


Long:  ['Parusmajor', 'Turduspilaris', 'Passerdomesticus', 'Luscinialuscinia', 'Dendrocoposmajor', 'Corvusmonedula', 'Phoenicurusphoenicurus', 'Erithacusrubecula', 'Picapica', 'Phoenicurusochruros', 'Garrulusglandarius', 'Passermontanus', 'Corvuscorone', 'Corvusfrugilegus', 'Coccothraustescoccothraustes', 'Sittaeuropaea', 'Alaudaarvensis', 'Streptopeliadecaocto', 'Phylloscopustrochilus', 'Delichonurbicum', 'Turdusphilomelos', 'Phylloscopuscollybita', 'Fringillacoelebs', 'Sturnusvulgaris', 'Emberizacitrinella', 'Columbapalumbus', 'Troglodytestroglodytes', 'Paruscaeruleus', 'Apusapus', 'Cardueliscarduelis', 'Chlorischloris', 'Motacillaalba', 'Turdusmerula'] 
Short:  ['0Parus', '1Passe', '2Lusci', '3Phoen', '4Erith', '5Phoen', '6Sitta', '7Alaud', '8Phyll', '9Turdu', '10Phyll', '11Fring', '12Sturn', '13Ember', '14Colum', '15Trogl', '16Cardu', '17Chlor', '18Turdu']


In [73]:

import shutil

counter=0
for birdNumber, bird in enumerate(birdsShort): # for each class (bird) check where the file should be copied
    print(counter)
    counter=0
    for setName, fileSet in zip(setNames, sets): # check for all datasets: train, val and test sests 
        for setNumber in fileSet[birdNumber]:
            
            for fileNumber, file in enumerate(flist[birdNumber]):
                if setNumber == fileNumber: # if file number to copy is same as number of file, then copy it
                    
                    for root, dirs, files in os.walk("../data/mels-19class/"):
                        for file2 in files:
                            if extractName(file) in file2:
                                counter=counter+1
                                source=root+"/"+file2
                                
                                destination = "data/19_class/"+setName+bird+"/"
                                if not os.path.exists(destination):
                                    os.makedirs(destination)
                                shutil.copy2(source, destination)
                                #print(source, "   ->   ", destination)

            

0
8577
1246
3107
2715
21116
1675
2117
2188
4481
35756
5615
10376
3117
9905
977
5226
762
1238
